<a href="https://colab.research.google.com/github/GhosttCasper/StyleTransferAndTelegramBot/blob/master/Telegram_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision
!pip3 install pillow==4.1.1

In [0]:
%matplotlib inline

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from scipy import misc

import torchvision.transforms as transforms
import torchvision.models as models
import copy

In [0]:
class ContentLoss(nn.Module):

        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            # we 'detach' the target content from the tree used
            # to dynamically compute the gradient: this is a stated value,
            # not a variable. Otherwise the forward method of the criterion
            # will throw an error.
            self.target = target.detach()#это константа. Убираем ее из дерева вычеслений
            self.loss = F.mse_loss(self.target, self.target )#to initialize with something

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input

In [0]:
def gram_matrix(input):
        batch_size, h, w, f_map_num = input.size()  # batch size(=1)
        # b=number of feature maps
        # (h,w)=dimensions of a feature map (N=h*w)

        features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(batch_size * h * w * f_map_num)

In [0]:
class StyleLoss(nn.Module):
        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()
            self.loss = F.mse_loss(self.target, self.target)# to initialize with something

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input

In [0]:
class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std

In [0]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

In [0]:
def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                                   style_img, content_img, device,
                                   content_layers=content_layers_default,
                                   style_layers=style_layers_default):
        cnn = copy.deepcopy(cnn)

        # normalization module
        normalization = Normalization(normalization_mean, normalization_std).to(device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                # The in-place version doesn't play very nicely with the ContentLoss
                # and StyleLoss we insert below. So we replace with out-of-place
                # ones here.
                #Переопределим relu уровень
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module("style_loss_{}".format(i), style_loss)
                style_losses.append(style_loss)

        # now we trim off the layers after the last content and style losses
        #выбрасываем все уровни после последенего styel loss или content loss
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        model = model[:(i + 1)]

        return model, style_losses, content_losses

In [0]:
def get_input_optimizer(input_img):
        # this line to show that input is a parameter that requires a gradient
        #добоваляет содержимое тензора катринки в список изменяемых оптимизатором параметров
        optimizer = optim.LBFGS([input_img.requires_grad_()]) 
        return optimizer

In [0]:
# В данном классе мы хотим полностью производить всю обработку картинок, которые поступают к нам из телеграма.
# Это всего лишь заготовка, поэтому не стесняйтесь менять имена функций, добавлять аргументы, свои классы и
# все такое.
class StyleTransferModel:

    def __init__(self):
        # Сюда необходимо перенести всю иницализацию, вроде загрузки свеерточной сети и т.д.
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.cnn = models.vgg19(pretrained=True).features.to(self.device).eval()
        self.cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(self.device)
        self.cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(self.device)

    def transfer_style(self, content_img_stream, style_img_stream, update, context, 
                       content_img_size, num_steps = 500, style_weight = 100000, content_weight=1):               
        # Этот метод по переданным картинкам в каком-то формате (PIL картинка, BytesIO с картинкой
        # или numpy array на ваш выбор). В телеграм боте мы получаем поток байтов BytesIO,
        # а мы хотим спрятать в этот метод всю работу с картинками, поэтому лучше принимать тут эти самые потоки
        # и потом уже приводить их к PIL, а потом и к тензору, который уже можно отдать модели.
        # В первой итерации, когда вы переносите уже готовую модель из тетрадки с занятия сюда нужно просто
        # перенести функцию run_style_transfer (не забудьте вынести инициализацию, которая
        # проводится один раз в конструктор.

        content_img = self.process_image(content_img_stream, content_img_size)          
        style_img = self.process_image(style_img_stream, content_img_size)          
        input_img = content_img.clone()
        # if you want to use white noise instead uncomment the below line:
        # input_img = torch.randn(content_img.data.size(), device=device)

        #self.send_debug_message_info_image(context, update, content_img_stream, content_img, style_img)

        model, style_losses, content_losses = get_style_model_and_losses(self.cnn,
            self.cnn_normalization_mean, self.cnn_normalization_std, style_img, content_img, self.device)
        optimizer = get_input_optimizer(input_img)

        print('Optimizing..')
        run = [0]
        while run[0] <= num_steps:

          def closure():
            # correct the values 
            # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            
            model(input_img)
            
            style_score = 0
            content_score = 0
            
            for sl in style_losses:
              style_score += sl.loss
            for cl in content_losses:
              content_score += cl.loss

            #взвешивание ощибки
            style_score *= style_weight
            content_score *= content_weight

            loss = style_score + content_score
            loss.backward()

            run[0] += 1
            return style_score + content_score
            
          optimizer.step(closure)

        # a last correction...
        input_img.data.clamp_(0, 1) 
        return self.tensor_to_PIL(input_img)
        # Сейчас этот метод просто возвращает не измененную content картинку
        # Для наглядности мы сначала переводим ее в тензор, а потом обратно
        #return misc.toimage(self.process_image(content_img_stream)[0])

    # В run_style_transfer используется много внешних функций, их можно добавить как функции класса
    # Если понятно, что функция является служебной и снаружи использоваться не должна, то перед именем функции
    # принято ставить _ (выглядит это так: def _foo() )
    # Эта функция тоже не является
    def process_image(self, img_stream, size):
        image = self.Bytes_stream_to_PIL(img_stream)
        return self.PIL_to_tensor(image, self.device, size)
    
    def Bytes_stream_to_PIL(self, img_stream):      
      img_stream.seek(0)
      image = Image.open(img_stream) # приведение потока байтов io.BytesIO к PIL изображению
      return image

    def PIL_to_tensor(self, image, device, size):
      loader = transforms.Compose([
              transforms.Resize(size),  # нормируем размер изображения
              transforms.CenterCrop(size),
              transforms.ToTensor()])  # превращаем в удобный формат
              
      image = loader(image).unsqueeze(0) # PIL_to_tensor
      return image.to(device, torch.float)
    
    def tensor_to_PIL(self, tensor):
      image = tensor.cpu().clone()
      #image = image.squeeze(0)
      unloader = transforms.ToPILImage()  # reconvert into PIL image    
      return unloader(image[0])

    def send_debug_message_info_image(self, context, update, content_img_stream, content_img, style_img):
      context.bot.send_message(chat_id=update.effective_chat.id, text="Building the style transfer model")   
      context.bot.send_message(chat_id=update.effective_chat.id, text="content_img_stream {}".format(content_img_stream))
      context.bot.send_message(chat_id=update.effective_chat.id, text="content_img {}".format(content_img))
      context.bot.send_message(chat_id=update.effective_chat.id, text="style_img {}".format(style_img))

In [11]:
!pip install python-telegram-bot --upgrade
!pip install tornado==4.5.1 distributed==1.21 dask-ml[complete]
#!pip install python-telegram-bot==4.0rc1

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (12.3.0)
Processing /root/.cache/pip/wheels/84/bf/40/2f6ef700f48401ca40e5e3dd7d0e3c0a90e064897b7fe5fc08/tornado-6.0.3-cp36-cp36m-linux_x86_64.whl
ERROR: google-colab 1.0.0 has requirement tornado~=4.5.0, but you'll have tornado 6.0.3 which is incompatible.
ERROR: dask-ml 1.2.0 has requirement distributed>=2.4.0, but you'll have distributed 1.21.0 which is incompatible.
  Found existing installation: tornado 4.5.1
    Uninstalling tornado-4.5.1:
      Successfully uninstalled tornado-4.5.1


Processing /root/.cache/pip/wheels/4c/f7/7d/479f75b6f637b44b9fc5749de49a5d21d8e3a8d5907e37fc6e/tornado-4.5.1-cp36-cp36m-linux_x86_64.whl
ERROR: python-telegram-bot 12.3.0 has requirement tornado>=5.1, but you'll have tornado 4.5.1 which is incompatible.
ERROR: dask-ml 1.2.0 has requirement distributed>=2.4.0, but you'll have distributed 1.21.0 which is incompatible.
  Found existing installation: tornado 6.0.3
    Uninstalling tornado-6.0.3:
      Successfully uninstalled tornado-6.0.3


In [12]:
import telegram
TOKEN = '833755366:AAFPZ05JsDq3aFcGn7WP59kXa60L7Dp-Iec'
bot = telegram.Bot(token=TOKEN)
print(bot.get_me())

{'id': 833755366, 'first_name': 'BotStyleTransfer', 'is_bot': True, 'username': 'StyleTransferPictureBot'}


In [0]:
from telegram.ext import Updater
from telegram.ext.dispatcher import run_async
updater = Updater(token=TOKEN, request_kwargs={'proxy_url': 'socks5h://163.172.152.192:1080'}, use_context=True)
dispatcher = updater.dispatcher

In [0]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
def build_menu(buttons,
               n_cols,
               header_buttons=None,
               footer_buttons=None):
    menu = [buttons[i:i + n_cols] for i in range(0, len(buttons), n_cols)]
    if header_buttons:
        menu.insert(0, [header_buttons])
    if footer_buttons:
        menu.append([footer_buttons])
    return menu

In [0]:
from telegram import ReplyKeyboardMarkup, KeyboardButton
from telegram.ext import Updater, CommandHandler, MessageHandler, filters
def start2(update, context):
    button_list = [
    KeyboardButton("Загрузить фото", callback_data=...),
    #KeyboardButton("col2", callback_data=...),
    #KeyboardButton("row 2", callback_data=...)
    ]
    reply_markup = ReplyKeyboardMarkup(button_list) #build_menu(button_list, n_cols=1)
    bot.send_message(update.message.chat_id, "I'm a bot, please send 2 pictures to me!", reply_markup=reply_markup)

In [0]:
def start(update, context):
    start_message_ru = "Я бот, пожалуйста, поговори со мной или отправь 2 фотографии - контент и стиль! \nКонтент - это фото, к которой будет применён стиль второй картинки."
    start_message_en = "I'm a bot, please talk to me or send content and style photos!"
    context.bot.send_message(chat_id=update.effective_chat.id, text=start_message_ru)

from telegram.ext import CommandHandler
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [0]:
def echo(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text=update.message.text)

from telegram.ext import MessageHandler, Filters
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)

In [0]:
from io import BytesIO

# В бейзлайне пример того, как мы можем обрабатывать две картинки, пришедшие от пользователя.

model = StyleTransferModel()
first_image_file = {}
first_image_info = {}

def send_prediction_on_photo(update, context):
    # Нам нужно получить две картинки, чтобы произвести перенос стиля, но каждая картинка приходит в
    # отдельном апдейте, поэтому в простейшем случае мы будем сохранять id первой картинки в память,
    # чтобы, когда уже придет вторая, мы могли загрузить в память уже сами картинки и обработать их.
    # Точно место для улучшения, я бы

    try:  
      chat_id = update.message.chat_id    
      print("Got image from {}".format(chat_id)) #context.bot.send_message(chat_id=update.effective_chat.id, text="Got image from {}".format(chat_id))

      # получаем информацию о картинке
      image_info = update.message.photo[-1]    
      image_file = bot.get_file(image_info)
      #send_debug_message(update, context, image_info, image_file, first_image_file)
      
      if chat_id in first_image_file:
          # первая картинка, которая к нам пришла станет content image, а вторая style image
          content_image_stream = BytesIO()
          first_image_file[chat_id].download(out=content_image_stream)    # загружаем первую content картинку
          first_image_size = [first_image_info[chat_id].height, first_image_info[chat_id].width]
          del first_image_file[chat_id]
          del first_image_info[chat_id]

          style_image_stream = BytesIO() 
          image_file.download(out=style_image_stream)

          message_second_picture ="Отлично! Применяю стиль к первой картинке. Сразу после выполнения отправлю результат!"
          context.bot.send_message(chat_id=update.effective_chat.id, text = message_second_picture)

          output = model.transfer_style(content_img_stream = content_image_stream, style_img_stream = style_image_stream, update=update, 
                                        context=context, content_img_size = first_image_size) # Построение модели transfer_style
          # теперь отправим назад фото
          output_stream = BytesIO()
          output.save(output_stream, format='PNG')
          output_stream.seek(0)
          message_result = "Готово!"
          context.bot.send_message(chat_id=update.effective_chat.id, text = message_result)
          bot.send_photo(chat_id, photo=output_stream)

          message_end = "Экспериментируй, получай новые картинки! Снова отправляй 2 фото!"
          context.bot.send_message(chat_id=update.effective_chat.id, text=message_end)
          print("Sent Photo to user")      
      else:
          first_image_file[chat_id] = image_file
          first_image_info[chat_id] = image_info
          message_first_picture = "Молодец! У тебя получилось отправить первую картинку. Теперь отправь ещё одну картинку, стиль которой будет применён к первой."
          context.bot.send_message(chat_id=update.effective_chat.id, text=message_first_picture)

    except Exception as inst:
      error_message = "Что-то пошло не так. Пожалуйста, попробуйте снова и отправьте сообщение об ошибке программисту @GhosttCasper"
      context.bot.send_message(chat_id=update.effective_chat.id, text=error_message)
      context.bot.send_message(chat_id=update.effective_chat.id, text="Ошибка: {}".format(type(inst))) # экземпляр исключения
      context.bot.send_message(chat_id=update.effective_chat.id, text="{}".format(inst.args))    # аргументы хранимые в .args
      context.bot.send_message(chat_id=update.effective_chat.id, text="{}".format(inst))         # __str__ позволяет вывести аргументы сразу

def send_debug_message(update, context, image_info, image_file, first_image_file):
    width = image_info.width
    height = image_info.height  
    context.bot.send_message(chat_id=update.effective_chat.id, text="image_info {}".format(image_info))    
    context.bot.send_message(chat_id=update.effective_chat.id, text="image_file {}".format(image_file))
    context.bot.send_message(chat_id=update.effective_chat.id, text="first_image_file {}".format(first_image_file))
    context.bot.send_message(chat_id=update.effective_chat.id, text="width {}".format(width))
    context.bot.send_message(chat_id=update.effective_chat.id, text="height {}".format(height))

In [0]:
from telegram.ext import MessageHandler, Filters
photo_handler = MessageHandler(Filters.photo, send_prediction_on_photo)
updater.dispatcher.add_handler(photo_handler)

In [0]:
def logs(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="{}".format(logging.getLogger(__name__)))
    context.bot.send_message(chat_id=update.effective_chat.id, text="{}".format(logging.getLogger))

log_handler = CommandHandler('logs', logs)
dispatcher.add_handler(log_handler)

In [0]:
def caps(update, context):
    text_caps = ' '.join(context.args).upper()
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_caps)

caps_handler = CommandHandler('caps', caps)
dispatcher.add_handler(caps_handler)

In [0]:
from telegram import InlineQueryResultArticle, InputTextMessageContent
def inline_caps(update, context):
    query = update.inline_query.query
    if not query:
        return
    results = list()
    results.append(
        InlineQueryResultArticle(
            id=query.upper(),
            title='Caps',
            input_message_content=InputTextMessageContent(query.upper())
        )
    )
    context.bot.answer_inline_query(update.inline_query.id, results)

from telegram.ext import InlineQueryHandler
inline_caps_handler = InlineQueryHandler(inline_caps)
dispatcher.add_handler(inline_caps_handler)

In [0]:
def unknown(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Sorry, I didn't understand that command.")

unknown_handler = MessageHandler(Filters.command, unknown)
dispatcher.add_handler(unknown_handler)

In [25]:
updater.start_polling()